In [1]:
import pandas as pd
from datetime import date, timedelta
import json

pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
state = 'West Virginia'
state_slug = state.replace(' ','-')
state_fips = '54'
today = str(date.today())
print("Today's date:", today)

cnty_join = pd.read_csv('../data/tl_2019_us_county.csv',dtype={'STATEFP':str,'COUNTYFP':str,'GEOID':str})
cnty_join = cnty_join.loc[cnty_join['STATEFP'] == state_fips]
cnty_join = cnty_join[['STATEFP','COUNTYFP','GEOID','NAMELSAD']]

case_data = pd.read_csv('../../data/case-data/nytimes/20200330-nytime-us-counties.csv',dtype={'fips':str})
state_cases = case_data.loc[case_data['state'] == state]
state_cases['date'] = pd.to_datetime(state_cases['date'], format='%Y-%m-%d', errors='ignore')

cases_by_day = state_cases.groupby('date').agg({'cases':'sum','deaths':'sum'}).reset_index()
most_recent = cases_by_day.sort_values('date',ascending=False).iloc[0]
most_recent_date = most_recent['date']

print('Total cases reported for',state,':',most_recent['cases'])
print('Total deaths reported for',state,':',most_recent['deaths'])
print('Most recent case confirmation date:',most_recent_date)
display(cases_by_day)

Today's date: 2020-03-31
Total cases reported for West Virginia : 145
Total deaths reported for West Virginia : 1
Most recent case confirmation date: 2020-03-30 00:00:00


/Users/akanik/.virtualenvs/covid19/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,date,cases,deaths
0,2020-03-17,1,0
1,2020-03-18,2,0
2,2020-03-19,5,0
3,2020-03-20,8,0
4,2020-03-21,12,0
5,2020-03-22,16,0
6,2020-03-23,16,0
7,2020-03-24,39,0
8,2020-03-25,51,0
9,2020-03-26,76,0


## County + day dataset

In addition to the above cases per county per day, we need to record county + days that have no cases or deaths so we can get a good time series.

In [3]:
covid_start = date(2020, 1, 21)   # start date
delta = most_recent_date.date() - covid_start       # as timedelta
delta

days = []
for i in range(delta.days + 1):
    day = covid_start + timedelta(days=i)
    days.append(day)
    
fullDateTime = pd.DataFrame(columns=['date','fips','county'])
for day in days:
    for index, row in cnty_join.iterrows():
        fullDateTime = fullDateTime.append({'date': day, 
                                            'fips': row['GEOID'], 
                                            'county': row['NAMELSAD']}, ignore_index=True)

fullDateTime['date'] = pd.to_datetime(fullDateTime['date'], format='%Y-%m-%d', errors='coerce')

fullDateTime['date_str'] = fullDateTime['date'].dt.strftime('%Y-%m-%d')
fullDateTime['join_field'] = fullDateTime['date_str'] + '-' + fullDateTime['fips']

by_county = state_cases
by_county['date_str'] = by_county['date'].dt.strftime('%Y-%m-%d')
by_county['join_field'] = by_county['date_str'] + '-' + by_county['fips']
    
fullDT_joined = fullDateTime.merge(by_county,how='left',on='join_field')
fullDT_joined = fullDT_joined.fillna(0)

fullDT_joined = fullDT_joined[['date_x','fips_x','county_x','cases','deaths']]
fullDT_joined.rename(columns={'date_x':'date','fips_x':'fips','county_x':'county'}, inplace=True)

fullDT_joined.to_csv('../app/assets/data/'+str(today)+'-'+state_slug+'-export.csv',index=False)

/Users/akanik/.virtualenvs/covid19/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/akanik/.virtualenvs/covid19/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
by_county.loc[by_county['fips'].isna()]

In [ ]:
by_county.groupby('date')['cases'].sum()

In [ ]:
fullDT_joined.groupby('date')['cases'].sum()